## Teste de Hipótese A/B Testing em Campanha de Marketing
 
 ## Contexto
 
 Empresas utilizam testes A/B para avaliar o impacto de campanhas de marketing.
 Neste estudo, temos dois grupos:
 
 - Grupo "ad": usuários expostos ao anúncio.
 - Grupo "psa": usuários não expostos (controle).
 
 O objetivo é verificar se a diferença observada nas taxas de conversão é estatisticamente significativa.

In [1]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
df = pd.read_csv('../data/marketing_AB.csv')

In [3]:
df['converted'] = df['converted'].astype(int)

In [4]:
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,0,130,Monday,20
1,1,1119715,ad,0,93,Tuesday,22
2,2,1144181,ad,0,21,Tuesday,18
3,3,1435133,ad,0,355,Tuesday,10
4,4,1015700,ad,0,276,Friday,14


## Pergunta de Negócio

A exposição ao anúncio aumenta significativamente a taxa de conversão?

## Formulação das Hipóteses

H₀ (Hipótese Nula):
A taxa de conversão é igual entre os grupos.

H₁ (Hipótese Alternativa):
A taxa de conversão é diferente entre os grupos.

Nível de significância:
α = 0.05

In [5]:
# Separar grupos
group_ad = df[df['test group'] == 'ad']
group_psa = df[df['test group'] == 'psa']

# Tamanhos das amostras
n_ad = len(group_ad)
n_psa = len(group_psa)

# Número de conversões
x_ad = group_ad['converted'].sum()
x_psa = group_psa['converted'].sum()

# Proporções amostrais
p_ad = x_ad / n_ad
p_psa = x_psa / n_psa

n_ad, n_psa, p_ad, p_psa

(564577,
 23524,
 np.float64(0.025546559636683747),
 np.float64(0.01785410644448223))

## Verificação das Condições para Aproximação Normal

Para aplicar o teste Z para proporções, é necessário que:

- n·p > 5
- n·(1-p) > 5

Como todas as quantidades n·p e n·(1−p) são significativamente maiores que 5,
a distribuição amostral da diferença das proporções pode ser aproximada
pela distribuição normal, justificando o uso do teste Z.

In [6]:
n_ad * p_ad, n_ad * (1 - p_ad), n_psa * p_psa, n_psa * (1 - p_psa)

(np.float64(14423.0),
 np.float64(550154.0),
 np.float64(420.0),
 np.float64(23104.0))

In [7]:
# Proporção combinada (pooled)
p_pool = (x_ad + x_psa) / (n_ad + n_psa)

p_pool

np.float64(0.02523886203220195)

In [8]:
# Erro padrão sob H0
se = np.sqrt(p_pool * (1 - p_pool) * ((1/n_ad) + (1/n_psa)))

se

np.float64(0.0010437410649006525)

In [9]:
# Estatística Z
z_score = (p_ad - p_psa) / se

z_score

np.float64(7.3700781265454145)

In [10]:
# p-valor (teste bilateral)
p_value = 2 * (1 - norm.cdf(abs(z_score)))

p_value

np.float64(1.7053025658242404e-13)

In [11]:
# %%
alpha = 0.05

if p_value < alpha:
    decision = "Rejeitamos H0"
else:
    decision = "Não rejeitamos H0"

decision

'Rejeitamos H0'

 ## Interpretação dos Resultados
 
 Com base no p-valor obtido e considerando α = 0.05:
 
 - Se p-valor < 0.05 → rejeitamos H₀ e concluímos que há evidência estatística de diferença entre os grupos.
 - Se p-valor ≥ 0.05 → não há evidência suficiente para afirmar que as taxas diferem.
 
 A significância estatística indica que a diferença observada dificilmente ocorreu por acaso.
 
 Entretanto, significância estatística não implica necessariamente relevância prática ou impacto financeiro.


In [12]:
diff = p_ad - p_psa

se_ci = np.sqrt((p_ad * (1 - p_ad)) / n_ad + 
                (p_psa * (1 - p_psa)) / n_psa)

ci_lower = diff - 1.96 * se_ci
ci_upper = diff + 1.96 * se_ci

ci_lower, ci_upper

(np.float64(0.00595090043017032), np.float64(0.009434005954232714))

## Intervalo de Confiança (95%)
 
 O intervalo de confiança representa a faixa de valores plausíveis para a diferença real das proporções.
 
 - Se o intervalo NÃO contém zero → evidência de diferença estatisticamente significativa.
 - Se o intervalo contém zero → não podemos afirmar diferença.
